In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, ExtraTreesClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

## Первый взгляд

In [2]:
train = pd.read_csv("C:\\Users\Я\Documents\P\mgf\contest_train.csv",  engine='python')

In [3]:
train.info()
train.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24521 entries, 0 to 24520
Columns: 262 entries, ID to FEATURE_259
dtypes: float64(260), int64(2)
memory usage: 49.0 MB


,ID,TARGET,FEATURE_0,FEATURE_1,FEATURE_2,FEATURE_3,FEATURE_4,FEATURE_5,FEATURE_6,FEATURE_7,...,FEATURE_250,FEATURE_251,FEATURE_252,FEATURE_253,FEATURE_254,FEATURE_255,FEATURE_256,FEATURE_257,FEATURE_258,FEATURE_259
0,1053371037,1,2.0,2.0,0.0,0.0,0.0,0.0,0.0,-114.527812,...,39.0,2.0,223.118594,290.015143,1.0,1.0,0.0,1.0,2.0,2.0
1,76507184,0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,329.828334,...,25.0,82.0,479.616045,611.651594,0.0,1.0,0.0,1.0,1.0,1.0
2,76753690,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.427338,...,119.0,0.0,352.268014,564.573421,1.0,1.0,0.0,1.0,1.0,1.0
3,1026167358,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,23.621397,...,1.0,0.0,410.524985,421.653876,1.0,0.0,0.0,1.0,2.0,2.0
4,7220095746,0,4.0,3.0,1.0,0.0,1.0,0.0,0.0,677.745861,...,21.0,0.0,439.171356,476.189288,1.0,1.0,0.0,1.0,1.0,1.0


## Разобъем датасет

In [4]:
X = train.drop(['TARGET', 'ID'], axis=1) 
y = train.TARGET 
X.head()

,FEATURE_0,FEATURE_1,FEATURE_2,FEATURE_3,FEATURE_4,FEATURE_5,FEATURE_6,FEATURE_7,FEATURE_8,FEATURE_9,...,FEATURE_250,FEATURE_251,FEATURE_252,FEATURE_253,FEATURE_254,FEATURE_255,FEATURE_256,FEATURE_257,FEATURE_258,FEATURE_259
0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,-114.527812,-17.218055,0.0,...,39.0,2.0,223.118594,290.015143,1.0,1.0,0.0,1.0,2.0,2.0
1,1.0,1.0,0.0,0.0,1.0,0.0,0.0,329.828334,NaN,3.0,...,25.0,82.0,479.616045,611.651594,0.0,1.0,0.0,1.0,1.0,1.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,17.427338,-6.680747,0.0,...,119.0,0.0,352.268014,564.573421,1.0,1.0,0.0,1.0,1.0,1.0
3,1.0,1.0,0.0,0.0,0.0,0.0,0.0,23.621397,36.659534,0.0,...,1.0,0.0,410.524985,421.653876,1.0,0.0,0.0,1.0,2.0,2.0
4,4.0,3.0,1.0,0.0,1.0,0.0,0.0,677.745861,NaN,3.0,...,21.0,0.0,439.171356,476.189288,1.0,1.0,0.0,1.0,1.0,1.0


# Предварительная обработка

### Найдем наиболее коррелирующие фичи:

In [5]:
corr_matrix = X.corr().abs() # матрица корреляции
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool)) # всё, что выше главной диагонали
most_correlated = [column for column in upper.columns if any(upper[column] > 0.95)]
most_correlated

['FEATURE_72',
 'FEATURE_73',
 'FEATURE_79',
 'FEATURE_84',
 'FEATURE_113',
 'FEATURE_119',
 'FEATURE_120',
 'FEATURE_153',
 'FEATURE_158',
 'FEATURE_164',
 'FEATURE_173',
 'FEATURE_186',
 'FEATURE_195',
 'FEATURE_200',
 'FEATURE_208',
 'FEATURE_225',
 'FEATURE_228',
 'FEATURE_230',
 'FEATURE_232',
 'FEATURE_233',
 'FEATURE_235',
 'FEATURE_237',
 'FEATURE_245',
 'FEATURE_254']

### и фичи с наибольшей потерей данных:

In [6]:
percent_missing = X.isnull().sum() * 100 / len(X)
missing_value_df = pd.DataFrame({'percent_missing': percent_missing})
missing_value_df.sort_values('percent_missing', inplace=True, ascending = False)
missing_value_df[:10]

,percent_missing
FEATURE_189,98.674605
FEATURE_194,70.327474
FEATURE_190,70.111333
FEATURE_191,62.432201
FEATURE_192,53.260471
FEATURE_193,51.368215
FEATURE_187,48.440113
FEATURE_71,12.071286
FEATURE_73,12.071286
FEATURE_195,11.365768


### В семи случаях - большая потеря данных

In [7]:
missing_features = ['FEATURE_190', 'FEATURE_194', 'FEATURE_189', 'FEATURE_191', 'FEATURE_192', 'FEATURE_193', 'FEATURE_187']

## Удалим сильно коррелирующие столбцы и столбцы с большой потерей данных:

In [8]:
to_drop = missing_features + most_correlated 
X.drop(to_drop, axis=1, inplace=True)

## Разобьем данные:

In [9]:
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0) 

## Заменим NaN с помощью SimpleImputer:

In [10]:
imputer = SimpleImputer() 

imputed_train_X = pd.DataFrame(imputer.fit_transform(train_X))
imputed_train_X.columns = train_X.columns

imputed_val_X = pd.DataFrame(imputer.fit_transform(val_X))
imputed_val_X.columns = val_X.columns

## и прошкалируем:

In [11]:
scaler = StandardScaler()
scaler.fit(imputed_train_X)
scale_train_X = scaler.transform(imputed_train_X)

scaler.fit(imputed_val_X)
scale_val_X = scaler.transform(imputed_val_X)

## Попробуем несколько классификаторов и выберем оптимальный.

In [12]:
def models_score(trainx, trainy, valx, valy):
    treemodel = DecisionTreeClassifier()# дерево
    treemodel.fit(trainx, trainy)
    tree_predictions = treemodel.predict(valx)
    tree_score = f1_score(valy, tree_predictions, average='macro')
    print('Decision Tree score:', tree_score)
    
    knn = KNeighborsClassifier() # метод ближайших соседей
    knn.fit(trainx, trainy)
    knn_pred = knn.predict(valx)
    knn_score = f1_score(valy, knn_pred, average='macro')
    print('KNN score:', knn_score)
    
    forest_model = RandomForestClassifier(random_state=0) # случайный лес
    forest_model.fit(trainx, trainy)
    forest_preds = forest_model.predict(valx)
    forest_score = f1_score(valy, forest_preds, average='macro')
    print('Random Forest score:', forest_score)
    
    gbclf = GradientBoostingClassifier(n_estimators=100) # градиентный буст
    gbclf.fit(trainx, trainy)
    gbclf_pred = gbclf.predict(valx)
    gbclf_score = f1_score(valy, gbclf_pred, average='macro')
    print('Gradient Boosting score:', gbclf_score)
    
    extratree = ExtraTreesClassifier(random_state=0) # экстра-деревья
    extratree.fit(trainx, trainy)
    extratree_pred = extratree.predict(valx)
    extratree_score = f1_score(valy, extratree_pred, average='macro')
    print('Extra Trees score:', extratree_score)
    
    adaboost = AdaBoostClassifier(random_state=0) #ada boost
    adaboost.fit(trainx, trainy)
    adaboost_pred = adaboost.predict(valx)
    adaboost_score = f1_score(valy, adaboost_pred, average='macro')
    print('Ada Boost score:', adaboost_score)
    
    xgbclf = XGBClassifier() # xgboost
    xgbclf.fit(trainx, trainy)
    xgb_pred = xgbclf.predict(valx)
    xgb_score = f1_score(valy, xgb_pred, average='macro')
    print('XGB score:', xgb_score)
    
    svmclf = svm.SVC() # Метод опорных векторов
    svmclf.fit(trainx, trainy)
    svm_pred = svmclf.predict(valx)
    svm_score = f1_score(valy, svm_pred, average='macro')
    print('SVM score:', svm_score)

In [13]:
models_score(scale_train_X, train_y, scale_val_X, val_y)

Decision Tree score: 0.42216311392635636
KNN score: 0.38628335778114214


C:\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Random Forest score: 0.39917864494544214
Gradient Boosting score: 0.40442536619892194


C:\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Extra Trees score: 0.4223333667074609
Ada Boost score: 0.4270277319051934
XGB score: 0.38149314011482344


C:\Anaconda\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVM score: 0.35417938835632046


## Попробуем уменьшить размерность с помощью метода главных компонент (РСА):

In [14]:
pca = PCA(0.9) 
pca.fit(scale_train_X)
pca_train_X = pca.transform(scale_train_X)
pca_val_X = pca.transform(scale_val_X)

## И посчитаем еще раз:

In [15]:
models_score(pca_train_X, train_y, pca_val_X, val_y)

Decision Tree score: 0.4169218156233208
KNN score: 0.38845454841240556


C:\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Random Forest score: 0.3661685884978614
Gradient Boosting score: 0.396561080347364


C:\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Extra Trees score: 0.35785611061247957
Ada Boost score: 0.4108784030799442
XGB score: 0.383303706972846


C:\Anaconda\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVM score: 0.36346118568224534


## Лучше всех себя показал AdaBoost без применения PCA. Будем использовать его.

# Тестовая выборка:

In [16]:
test = pd.read_csv("C:\\Users\Я\Documents\P\mgf\contest_test.csv",  engine='python')
ids = test['ID'].values
test.drop(['ID'] + to_drop + missing_features, axis=1, inplace=True)

## Аналогично заполним потери данных и прошкалируем:

In [17]:
imputed_test_X = pd.DataFrame(imputer.fit_transform(test))
imputed_test_X.columns = test.columns
scaler.fit(imputed_test_X)
scale_test_X = scaler.transform(imputed_test_X)

## Модель:

In [18]:
adaboost = AdaBoostClassifier(random_state=0)
adaboost.fit(scale_train_X, train_y)
adaboost_pred = adaboost.predict(scale_test_X)

## Выгрузим в файл:

In [19]:
output = pd.DataFrame({'ID': ids, 'TARGET': adaboost_pred})
output.to_csv("contest_answer.csv", index=False, sep=",")

# Второе задание: 

## Для отбора записей, степень уверенности модели которых максимальна, воспользуемся predict_proba:

In [20]:
adaboost_probas = adaboost.predict_proba(scale_test_X)
probs_df = pd.DataFrame(adaboost_probas)
probs_df.head()

,0,1,2
0,0.344789,0.333262,0.321949
1,0.348719,0.340570,0.310710
2,0.352257,0.321147,0.326596
3,0.346394,0.339137,0.314469
4,0.344511,0.338538,0.316951


## Пробежимся по всему датафрейму и оставим в строках только максимумы, остальное заменим нулями.

In [21]:
for index, row in probs_df.iterrows():
    for i,prob in enumerate(row):
        if prob != max(row): 
            probs_df.loc[index, i] = 0

## Вставим ID:

In [24]:
probs_df['ID'] = ids
probs_df.head(10)

,0,1,2,ID
0,0.344789,0.0,0.000000,1154270440
1,0.348719,0.0,0.000000,1147987574
2,0.352257,0.0,0.000000,1129622364
3,0.346394,0.0,0.000000,619797496
4,0.344511,0.0,0.000000,7391484886
5,0.343760,0.0,0.000000,10165219189
6,0.345737,0.0,0.000000,741382912
7,0.343278,0.0,0.000000,1130214814
8,0.342913,0.0,0.000000,78018389
9,0.000000,0.0,0.338026,75592880


## Затем отсортируем по каждому признаку и возьмем срез:

In [25]:
segment_1 = probs_df.sort_values(1, ascending = False)[:500]
segment_1_output = pd.DataFrame({'ID': segment_1.ID, 'TARGET': 1})

segment_2 = probs_df.sort_values(2, ascending = False)[:200]
segment_2_output = pd.DataFrame({'ID': segment_2.ID, 'TARGET': 2})

## Объединим и выгрузим в файл:

In [26]:
segments_output = pd.concat([segment_1_output, segment_2_output], ignore_index=True)
segments_output.to_csv("contest_segments.csv", index=False, sep=",")

### P.S.: Про сегменты ничего более оптимального не придумал, перечитал документацию pandas - решил в лоб. 